### ------------------------------------------------------------------------------------------------------------

In [ ]:
import os

if 'COLAB_GPU' in os.environ:
    !git clone https://github.com/impulsecorp/PickStocks.git
    !mv PickStocks/*.py .
    !mv PickStocks/data .
    !pip install -U -qq -r PickStocks/requirements.txt

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import system
from system import *
# small hack to prevent Colab error
try:
    from datablock import *
except:
    from datablock import *
seed

In [ ]:
%pylab inline

### Setup

In [ ]:
data_timeperiod = 'D'
data = get_data('SPY', period=data_timeperiod, nrows=None)
data = procdata_lite(data)

In [ ]:
# for inspectiion
print(data.shape)
data.head()

In [ ]:
system.train_set_end = 0.7 # percentage point specifying the training set end point (1.0 means all data is training set)
system.val_set_end = 1.0    # percentage point specifying the validation set end point (1.0 means no test set)
# basically this is the data with the values above, which are like sliders determining the layout
# [|0.0| ................... train .............. |0.7| ....................... val ..................... |1.0| no test ]
system.balance_data = 1

### ------------------------------------------------------------------------------------------------------------

### LogisticRegression

In [ ]:
# Train LogisticRegression classifier on train data
clf, scaler = train_classifier(LogisticRegression, data)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### XGBClassifier

In [ ]:
# Train XGBClassifier classifier on train data
clf, scaler = train_classifier(XGBClassifier, data)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### LGBMClassifier

In [ ]:
# Train LGBMClassifier classifier on train data
clf, scaler = train_classifier(LGBMClassifier, data)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### RandomForestClassifier

In [ ]:
# Train RandomForestClassifier classifier on train data
clf, scaler = train_classifier(RandomForestClassifier, data, n_jobs=-1)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### ExtraTreesClassifier

In [ ]:
# Train ExtraTreesClassifier classifier on train data
clf, scaler = train_classifier(ExtraTreesClassifier, data)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### CatBoostClassifier

In [ ]:
# Train CatBoostClassifier classifier on train data
clf, scaler = train_classifier(CatBoostClassifier, data, verbose=False)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### ------------------------------------------------------------------------------------------------------------